## Installations and Requirements

In [ ]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy

# Imports

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plot
from PIL import Image

# Read Image

### Read image.jpg from the folder (using OpenCV)

In [ ]:
image_bgr = cv2.imread('./image.jpg')

### Convert it to Gray (using OpenCV)

In [ ]:
image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)

# Descriptor Representations & Operations

### Implement HOG Descriptor (Using OpenCV)

### Implement HOG Descriptor (From Scratch Without OpenCV)